<a href="https://colab.research.google.com/github/alphoenixbiz/AlphaCode-EXPLORE/blob/master/Submissions/Classification%20Model/Alfred_predict03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
import pickle
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
%matplotlib inline
from sklearn.utils import resample
import re
import nltk
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score

### Install SQL Alchemy

In [41]:
!pip install sqlalchemy
!pip install pymssql

### Import packages

In [0]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine

### Create a connection to our database

In [0]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Alfred'

### Create connection

In [0]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

### What is the sql query we want to perform?

In [0]:
sql_query1 = "USE Alfred Select * from Bank_Statments"

 ### Using pandas we now take our connection and our query to create a dataframe

In [46]:
#import the training data
train = pd.read_sql_query(sql_query1, connection)
train.head()

,Transaction_Date,Posting_Date,Description,Debits,Credits,Category,column7,CATEGORIES,column9
0,2020-02-10,2020-02-10,MTN DATA BUNDLES,499,None,COMMUNICATION,None,HOME,None
1,2020-02-08,2020-02-09,APPLE.COM/BILL CORK IE,74.99,None,ENTERTAINMENT & EAT OUT,None,FOOD,None
2,2020-02-07,2020-02-09,PARKHURST HARDWARE RE PARKHURST ZA,75,None,HOME,None,TRANSPORT,None
3,2020-02-07,2020-02-08,SEATTLE RENNIES HOUSE JOHANNESBURG ZA,31.5,None,ENTERTAINMENT & EAT OUT,None,LIFE AND HEALTH,None
4,2020-02-07,2020-02-08,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,29.98,None,ENTERTAINMENT & EAT OUT,None,COMMUNICATION,None


In [0]:
# #import the test data
# test = pd.read_sql_query(sql_query2, connection)
# test.head()

# EDA

In [48]:
# get the target variable
# create our target variables
y=train.Category.apply(lambda x: x.lower())
y.head()


0              communication
1    entertainment & eat out
2                       home
3    entertainment & eat out
4    entertainment & eat out
Name: Category, dtype: object

In [0]:
features = train.Description.values
features2 = train.Description.values

In [50]:
features[:5]

array(['MTN DATA BUNDLES', 'APPLE.COM/BILL CORK IE',
       'PARKHURST HARDWARE    RE PARKHURST ZA',
       'SEATTLE RENNIES HOUSE JOHANNESBURG ZA',
       'FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA'], dtype=object)

# Text to Features

In [51]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

lem = WordNetLemmatizer()
# ps = PorterStemmer()
corpus =[]
# cleaning text
for i in range(0,len(features)):

    item = re.sub('[^a-zA-Z]', ' ',str(features)) 
    item = item.lower()
    item = item.split()
    stop_wrds = set(stopwords.words('english'))
    item = [lem.lemmatize(word, "v") for word in item if not word in stop_wrds]
    item = ' '.join(item)
    corpus.append(item)

new_features = corpus

In [53]:
new_features[:5]

['mtn data bundle apple com bill cork ie parkhurst hardware parkhurst za seattle rennies house johannesburg za flm rennie house eater braamfontein za flm rennie house eater braamfontein za flm rennie house eater braamfontein za flm rennie house eater braamfontein za seattle rennies house johannesburg za flm rennie house eater braamfontein za property rondebosch za seattle rennies house johannesburg za flm rennie house eater braamfontein za flm rennie house eater braamfontein za investecpb apple com bill itunes com ie bloom coffee studio cape town za seattle rennies house johannesburg za flm rennie house eater braamfontein za investecpb admed admed taxassist netcash seattle rennies house johannesburg za flm rennie house eater braamfontein za flm rennie house eater braamfontein za bolton rd collection parkview za virgin act rent levy mtn sp disc prem discovery vir atl vir atl gb amzn digital ch w kv us foundry cafe parktown nort za seattle rennies house johannesburg za flm rennie house e

In [54]:

# Create Ranked Statistical Features with TfidfVectorizer, best choice because insignificant words recieve a lower rank
obj = TfidfVectorizer()
final_features = obj.fit_transform(new_features)
print(final_features[0])

  (0, 119)	0.0025916913455350794
  (0, 1)	0.0025916913455350794
  (0, 376)	0.005183382691070159
  (0, 138)	0.005183382691070159
  (0, 334)	0.0025916913455350794
  (0, 291)	0.0025916913455350794
  (0, 182)	0.005183382691070159
  (0, 201)	0.0025916913455350794
  (0, 280)	0.0025916913455350794
  (0, 265)	0.0025916913455350794
  (0, 88)	0.005183382691070159
  (0, 162)	0.0025916913455350794
  (0, 198)	0.0025916913455350794
  (0, 264)	0.0025916913455350794
  (0, 429)	0.0025916913455350794
  (0, 93)	0.0025916913455350794
  (0, 95)	0.005183382691070159
  (0, 50)	0.005183382691070159
  (0, 375)	0.007775074036605239
  (0, 145)	0.007775074036605239
  (0, 259)	0.010366765382140318
  (0, 17)	0.010366765382140318
  (0, 408)	0.005183382691070159
  (0, 189)	0.005183382691070159
  (0, 186)	0.005183382691070159
  :	:
  (0, 67)	0.015550148073210477
  (0, 26)	0.007775074036605239
  (0, 194)	0.0025916913455350794
  (0, 193)	0.005183382691070159
  (0, 341)	0.0025916913455350794
  (0, 312)	0.0025916913455350

In [0]:

# split the train set to create a validation set
X_train, X_test, y_train, y_test = train_test_split(final_features, y, test_size=0.30)


In [0]:
#import model
from sklearn.neighbors import KNeighborsClassifier
cls = KNeighborsClassifier()

In [57]:
# train initial model
cls.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
# Making the predictions
pred1 = cls.predict(X_test)

In [59]:
# performance matrics and model eveluation using sklearn.metrics inbuilt classification metric
print(classification_report(y_test, pred1))

                         precision    recall  f1-score   support

          communication       0.00      0.00      0.00        26
              education       0.00      0.00      0.00         3
entertainment & eat out       0.00      0.00      0.00        48
                   fees       0.00      0.00      0.00        59
                   food       0.00      0.00      0.00        12
      gifts & donations       0.02      1.00      0.04         6
                   home       0.00      0.00      0.00        10
        life and health       0.00      0.00      0.00         8
                  other       0.00      0.00      0.00         7
  saving and investment       0.00      0.00      0.00         3
      short-term credit       0.00      0.00      0.00        10
              transport       0.00      0.00      0.00        91

               accuracy                           0.02       283
              macro avg       0.00      0.08      0.00       283
           weighted avg

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# submission

In [0]:
# save predictions
submission = pd.DataFrame({'description' : features2[X_train.shape[0]:],'category' : pred1.ravel()}) 

In [75]:
submission.head()

,description,category
0,VEE 401509249,gifts & donations
1,BUDGET2019,gifts & donations
2,SPOTIFYZA 5326*9249 20 DEC,gifts & donations
3,VOD PREPAID 0825197505,gifts & donations
4,FEE - PRE-PAID TOP UP,gifts & donations


In [0]:
# submission.to_sql('NLP_model_results', con=connection, if_exists='append', index=False,method='multi',chunksize=500)
